Read the json file as dataframe

In [29]:
import pandas as pd
import json
import numpy as np
import apriori


In [8]:
data_df = pd.read_json("data/music_app/data.json", orient = 'records', lines = True)

In [9]:
data_df.head()

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length
0,1538352180000,9,8,NextSong,Logged In,PUT,200,free,79,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Five Iron Frenzy,Canada,236.09424
1,1538352394000,30,29,NextSong,Logged In,PUT,200,paid,51,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Adam Lambert,Time For Miracles,282.82730
2,1538352416000,9,8,NextSong,Logged In,PUT,200,free,80,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Enigma,Knocking On Forbidden Doors,262.71302
3,1538352676000,30,29,NextSong,Logged In,PUT,200,paid,52,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Daft Punk,Harder Better Faster Stronger,223.60771
4,1538352678000,9,8,NextSong,Logged In,PUT,200,free,81,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,The All-American Rejects,Don't Leave Me,208.29995


Check if there are rare appearance of users,etc.

In [11]:
data_df_columns = data_df.columns.drop(['location', 'userAgent', 'artist', 'song', 'length', 'ts'])
for col in data_df_columns:
    print(col)
    print(data_df[col].value_counts())

userId
39        9632
          8346
92        7230
140       6880
300011    5732
          ... 
300003      35
100011      23
125         11
135          6
156          6
Name: userId, Length: 226, dtype: int64
sessionId
1840    1322
316     1247
236      905
635      890
368      799
        ... 
1286       1
1890       1
1374       1
2399       1
2426       1
Name: sessionId, Length: 2354, dtype: int64
page
NextSong                     228107
Home                          14457
Thumbs Up                     12551
Add to Playlist                6526
Add Friend                     4277
Roll Advert                    3933
Login                          3241
Logout                         3226
Thumbs Down                    2546
Downgrade                      2055
Help                           1726
Settings                       1514
About                           924
Upgrade                         499
Save Settings                   310
Error                           258
Submit Upg

Delete rows that belong to one session only:

In [12]:
print(data_df.shape)
data_df = data_df[data_df.sessionId > 1]
print(data_df.shape)

Group by __userId__ and __sessionId__

In [23]:

data_grouped = data_df.groupby(['userId','sessionId']).count()
user_session = list(data_grouped[data_grouped['ts'] < 5].index)
user_session_list = []
for item in user_session:
    if(item[0] != ''):
        user_session_list.append(list(item))
print(len(user_session_list))

231


Collect all consequences of songs for a user in one session:

In [24]:
artist_song = []
for user, session in user_session_list:
    selected = data_df[(data_df['userId'] == str(user)) & (data_df['sessionId'] == session) 
                       & (data_df['artist'].notna())]
    selected_list = list(selected[['artist','song']].values)
    if(len(selected_list) > 1):
        artist_song.append([str(l[0]) + "_" + str(l[1]) for l in selected_list])
print(artist_song)

[['Prince & The New Power Generation [with Eric Leeds on Flute]_Gett Off', 'Eels_Love Of The Loveless'], ['Black Eyed Peas_Gone Going', 'Paramore_All We Know (Album Version)'], ['Black Eyed Peas_I Gotta Feeling (FMIF Remix Edit)', 'John Waite_Missing You'], ['Radiohead_Creep (Explicit)', 'Richard Marx_Right Here Waiting (Single Edit)', 'Evanescence_Bring Me To Life', 'Nada Surf_Weightless'], ['Catherine Wheel_Wish You Were Here', "Kicking K8_Don't Let Me Fall"], ['Angels and Airwaves_The Gift', 'Chromeo_Night By Night'], ['Everything But The Girl_Better Things', 'The Killers_Losing Touch', 'Daft Punk_Something About Us'], ['M2M_Pretty Boy (LP Version)', 'Edward Sharpe & The Magnetic Zeros_Up From Below'], ['Lily Allen_Not Fair', 'Ornette Coleman Trio_Snowflakes And Sunshine (Live) (2002 Digital Remaster) (The Rudy Van Gelder Edition)'], ['Mano Negra_Mala Vida', 'Neutral Milk Hotel_Holland_ 1945'], ['Justice_DVNO', 'Jack Johnson_What You Thought You Need'], ['TiÃ\x83Â«sto feat. Calvin H

Create a variable to look for songs consequences: each song is located according to the appearance (1st - 0 column, 2nd - 1st column, etc..)

In [25]:
sequences_4 = pd.get_dummies(pd.DataFrame(artist_song))

Following we are detecting the frequent items appearing after each other, the place in the consequence is considered. Later the consequences with the high lift are shown. These are combinations appearing more often than other combinations with one of the songs from a combination. 

In [30]:
freq_items = apriori.detect_freq_items(sequences_4, max_len = 4)

Processing 1560 combinations | Sampling itemset size 4


In [31]:
association_rules = apriori.get_association_rules(freq_items, min_confidence = 0)
association_rules.lift.value_counts()

67.0    502
33.5     24
Name: lift, dtype: int64

In [36]:
association_rules[association_rules.lift==67]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,'0_A Skylit Drive_Running With The Light','1_Commodores_Nightshift',0.014925,0.014925,0.014925,1.0,67.0,0.014703,inf
1,'1_Commodores_Nightshift','0_A Skylit Drive_Running With The Light',0.014925,0.014925,0.014925,1.0,67.0,0.014703,inf
2,'1_A-Ha_What There Is','0_A Tribe Called Quest_Find A Way',0.014925,0.014925,0.014925,1.0,67.0,0.014703,inf
3,'0_A Tribe Called Quest_Find A Way','1_A-Ha_What There Is',0.014925,0.014925,0.014925,1.0,67.0,0.014703,inf
4,'0_ATB_The Summer','1_Rihanna_Rehab',0.014925,0.014925,0.014925,1.0,67.0,0.014703,inf
...,...,...,...,...,...,...,...,...,...
521,"'2_The xx_Basic Space', '0_Tiny Vipers_They Mi...","'1_Bread_Too Much Love', '3_Eminem_Mockingbird'",0.014925,0.014925,0.014925,1.0,67.0,0.014703,inf
522,'1_Bread_Too Much Love',"'3_Eminem_Mockingbird', '2_The xx_Basic Space'...",0.014925,0.014925,0.014925,1.0,67.0,0.014703,inf
523,'3_Eminem_Mockingbird',"'1_Bread_Too Much Love', '2_The xx_Basic Space...",0.014925,0.014925,0.014925,1.0,67.0,0.014703,inf
524,'2_The xx_Basic Space',"'1_Bread_Too Much Love', '3_Eminem_Mockingbird...",0.014925,0.014925,0.014925,1.0,67.0,0.014703,inf


Show all combinations of songs if users starts the session with Radiohead_Creep (Explicit)

In [33]:
association_rules[association_rules.antecedents == frozenset({'0_Radiohead_Creep (Explicit)'})]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
144,(0_Radiohead_Creep (Explicit)),(1_LMFAO_Yes),0.029851,0.014925,0.014925,0.5,33.5,0.01448,1.970149
147,(0_Radiohead_Creep (Explicit)),(1_Richard Marx_Right Here Waiting (Single Edit)),0.029851,0.014925,0.014925,0.5,33.5,0.01448,1.970149
149,(0_Radiohead_Creep (Explicit)),(2_Evanescence_Bring Me To Life),0.029851,0.014925,0.014925,0.5,33.5,0.01448,1.970149
150,(0_Radiohead_Creep (Explicit)),(3_Nada Surf_Weightless),0.029851,0.014925,0.014925,0.5,33.5,0.01448,1.970149
402,(0_Radiohead_Creep (Explicit)),(1_Richard Marx_Right Here Waiting (Single Edi...,0.029851,0.014925,0.014925,0.5,33.5,0.01448,1.970149
408,(0_Radiohead_Creep (Explicit)),(1_Richard Marx_Right Here Waiting (Single Edi...,0.029851,0.014925,0.014925,0.5,33.5,0.01448,1.970149
414,(0_Radiohead_Creep (Explicit)),"(2_Evanescence_Bring Me To Life, 3_Nada Surf_W...",0.029851,0.014925,0.014925,0.5,33.5,0.01448,1.970149
509,(0_Radiohead_Creep (Explicit)),(1_Richard Marx_Right Here Waiting (Single Edi...,0.029851,0.014925,0.014925,0.5,33.5,0.01448,1.970149
